In [2]:
import pdfplumber
import pytesseract
from pytesseract import Output
from PIL import Image

# Path to the scanned PDF
pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\pdf_files\sample_sensitive_pdf.pdf"

# Open the PDF with pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        image = page.to_image(resolution=500).original
        page_number = i + 1
        words_info = []
        # img = Image.open(image)
        custom_confg = r'--oem 3 --psm 6'
        ocr_data = pytesseract.image_to_data(image, output_type=Output.DICT, lang='eng')
        text = pytesseract.image_to_string(image, config=custom_confg, lang='eng')
        word_info = []
        for i in range(len(ocr_data['text'])):
            word_info = {
                'text': ocr_data['text'][i],
                'start_x': ocr_data['left'][i] * (page.width / image.width),
                'start_y': ocr_data['top'][i] * (page.height / image.height),
                'end_x': (ocr_data['left'][i] + ocr_data['width'][i]) * (page.width / image.width),
                'end_y': (ocr_data['top'][i] + ocr_data['height'][i]) * (page.height / image.height),
                'page_number': page_number
            }
            words_info.append(word_info)

In [3]:
from presidio_analyzer import AnalyzerEngineProvider
from presidio_anonymizer import  AnonymizerEngine
conf_path = r'C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\config\all-config-C4.yaml'
provider = AnalyzerEngineProvider(
    analyzer_engine_conf_file=conf_path
)
analyzer = provider.create_engine()
analyzer_result = analyzer.analyze(text=text,language='en')

anonimyzer = AnonymizerEngine()
result = anonimyzer.anonymize(text=text, analyzer_results=analyzer_result)



In [4]:
from collections import defaultdict
original_words = set(text.split())
new_words = set(result.text.split())
differing_words = original_words.difference(new_words)
word_data_map = defaultdict(list)
for word_data in words_info:
    word_data_map[word_data['text']].append(word_data)
differing_words_data = [data for word in differing_words for data in word_data_map[word]]



In [5]:
print(text)

Hello, my name is John Doe.

You can reach me at john.doe @example.com or at my phone number +1-555-123-4567.

My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street,
Springfield.
| have diabetes, and asthma. | vote for Democratic party and | am ecologist.

Best regards, John.



In [6]:
print(result.text)

<GREET>, my name is <PERSON>.

You can reach me at john.doe @example.com or at my phone number <NRP>-123-4567.

My credit card number is <DATE_TIME> and my home address is <DATE_TIME> Elm Street,
<LOCATION>.
| have diabetes, and asthma. | vote for <NRP> party and | am ecologist.

Best regards, <PERSON>.



In [7]:
words_info

[{'text': '',
  'start_x': 0.0,
  'start_y': 0.0,
  'end_x': 2356.0,
  'end_y': 3136.0,
  'page_number': 1},
 {'text': '',
  'start_x': 105.97824226867132,
  'start_y': 123.98273487005234,
  'end_x': 655.0204131524264,
  'end_y': 168.04628524198733,
  'page_number': 1},
 {'text': '',
  'start_x': 105.97824226867132,
  'start_y': 123.98273487005234,
  'end_x': 655.0204131524264,
  'end_y': 168.04628524198733,
  'page_number': 1},
 {'text': '',
  'start_x': 105.97824226867132,
  'start_y': 123.98273487005234,
  'end_x': 655.0204131524264,
  'end_y': 168.04628524198733,
  'page_number': 1},
 {'text': 'Hello,',
  'start_x': 105.97824226867132,
  'start_y': 123.98273487005234,
  'end_x': 209.94059405940595,
  'end_y': 164.01432638442463,
  'page_number': 1},
 {'text': 'my',
  'start_x': 227.93961618384063,
  'start_y': 134.06263201395905,
  'end_x': 282.94462779611297,
  'end_y': 168.04628524198733,
  'page_number': 1},
 {'text': 'name',
  'start_x': 297.05586114166977,
  'start_y': 135.070

In [11]:
print(result.text)

<GREET>, my name is <PERSON>.

You can reach me at john.doe @example.com or at my phone number <NRP>-123-4567.

My credit card number is <DATE_TIME> and my home address is <DATE_TIME> Elm Street,
<LOCATION>.
| have diabetes, and asthma. | vote for <NRP> party and | am ecologist.

Best regards, <PERSON>.



In [ ]:
pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\pdf_files\sample_sensitive_pdf.pdf"

output_pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\pdf_files\sample_sensitive_pdf_redacted.pdf"

from reportlab.pdfgen import canvas
from PyPDF2 import PdfWriter, PdfReader
temp_pdf_path = "temp_overlay.pdf"
# Step 1: Create a canvas for the overlay PDF
with pdfplumber.open(pdf_path) as pdf:
    overlay = canvas.Canvas(temp_pdf_path)

    for page_num, page in enumerate(pdf.pages, start=1):
        page_width = page.width
        page_height = page.height
        overlay.setPageSize((page_width, page_height))

        page_words = [word for word in differing_words_data if word['page_number'] == page_num]
        for word_data in page_words:
            x0, y0, x1, y1 = word_data['start_x'], word_data['start_y'], word_data['end_x'], word_data['end_y']
            overlay.setFillColor('black')
            overlay.setStrokeColor('black')
            overlay.setLineWidth(0.5)
            overlay.rect(x0, page_height - y1, x1 - x0, y1 - y0, fill=1)

        overlay.showPage()

    overlay.save()

input_pdf = PdfReader(pdf_path)
overlay_pdf = PdfReader(temp_pdf_path)
writer = PdfWriter()

for i, page in enumerate(input_pdf.pages):
    if i < len(overlay_pdf.pages):
        page.merge_page(overlay_pdf.pages[i])
    writer.add_page(page)
    
with open(output_pdf_path, "wb") as output_file:
    writer.write(output_file)

[{'text': 'John.', 'start_x': 375.9635741351913, 'start_y': 638.0574892092937, 'end_x': 480.06991810292146, 'end_y': 673.0491321517127, 'page_number': 1}, {'text': 'Springfield.', 'start_x': 107.994132746608, 'start_y': 429.97961245293413, 'end_x': 317.93472680601394, 'end_y': 473.03517311047847, 'page_number': 1}, {'text': 'Hello,', 'start_x': 105.97824226867132, 'start_y': 123.98273487005234, 'end_x': 209.94059405940595, 'end_y': 164.01432638442463, 'page_number': 1}, {'text': '+1-555-123-4567.', 'start_x': 1490.0310475491995, 'start_y': 235.00560198365324, 'end_x': 1857.9310597726442, 'end_y': 268.9892552116815, 'page_number': 1}, {'text': '4111-1111-1111-1111', 'start_x': 667.9797090820193, 'start_y': 335.9485719533474, 'end_x': 1099.9562400684513, 'end_y': 370.94021489576636, 'page_number': 1}, {'text': 'John', 'start_x': 460.0550055005501, 'start_y': 128.01469372761503, 'end_x': 552.0660066006601, 'end_y': 161.9983469556433, 'page_number': 1}, {'text': 'Democratic', 'start_x': 86